In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import category_encoders as ce

In [ ]:
df = pd.read_excel('/kaggle/input/date-fruit-datasets/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx')

using pandas to understand data


In [ ]:
df.info()
df['Class'].unique()
df.describe()
df.shape
df.groupby('Class').agg({'AREA':'mean', 'PERIMETER':'mean'})
lst = [x for x in df.columns if x != "Class"]
df[lst].corr()
df['Class'].value_counts()

data visualisation


In [ ]:
counts, bins, _ = plt.hist(df["AREA"], bins =50, color='skyblue', edgecolor = "black")
plt.xlabel('Значения')
plt.ylabel('Частота')
plt.title('Гистограмма данных')
plt.grid(axis='x', linestyle='--', alpha=1)
plt.grid(axis='y', linestyle='--', alpha=1)
# Находим координаты вершин столбцов
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# Рисуем линию по верхам столбцов
plt.plot(bin_centers, counts, color='red', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
sns.boxplot(df["PERIMETER"])

In [ ]:
sns.heatmap(df[lst].corr(), annot=True, cmap='viridis')
plt.title('Тепловая карта корреляций')
plt.show()

implementing binary encoder

In [ ]:
binary_encoder = ce.BinaryEncoder(cols=['Class'])
df_encoded = binary_encoder.fit_transform(df)

data splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class', axis=1), df_encoded[['Class_0', 'Class_1', 'Class_2']], test_size=0.33, random_state=42)

**model implementation**: so i have to classify dates. there are a lot of methods to do it, but i decided to use GradientBoosting

In [ ]:
XGB = XGBClassifier(n_estimators= 100, learning_rate = 0.1, max_depth = 3)
y_preds = XGB.fit(X_train, y_train).predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
accuracy = accuracy_score(y_test, y_preds)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
precision_score = precision_score(y_test, y_preds, average='micro')
print(f'precision_score: {precision_score}')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
recall_score = recall_score(y_test, y_preds, average='micro')
print(f'recall_score: {recall_score}')

In [ ]:
y_preds = pd.DataFrame(y_preds)
y_preds[0]